In [1]:
import csv
import json
import requests
import os

/Users/julia.cave.arbanas/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
endpoint = 'https://www.loc.gov/free-to-use'
parameters = {
    'fo' : 'json'
}

In [3]:
collection = 'wpa-posters'

In [4]:
collection_list_response = requests.get(endpoint + '/' + collection, params=parameters)

In [5]:
collection_json = collection_list_response.json()

local_json_collection = '/Users/julia.cave.arbanas/si676-2024-data/collection_project_WPA/collection_list_response.json'

with open(local_json_collection, 'w', encoding='utf-8', newline='') as f:
    json.dump(collection_json, f, ensure_ascii=False, indent=4)
print(f'JSON data saved')

JSON data saved


In [6]:
collection_json.keys()

dict_keys(['breadcrumbs', 'content', 'content_is_post', 'description', 'disable_max_line_length', 'expert_resources', 'manifest', 'next', 'next_sibling', 'options', 'pages', 'portal', 'previous', 'previous_sibling', 'site_type', 'timestamp', 'title', 'type'])

In [7]:
for k in collection_json['content']['set']['items']:
    print(k)

{'image': '/static/portals/free-to-use/public-domain/wpa-posters/a-book-mark-would-be-better-loc_6629878315_o.jpg', 'link': '/item/2011645389/', 'title': 'A book mark would be better!'}
{'image': '/static/portals/free-to-use/public-domain/wpa-posters/live-here-at-low-rent---lakeview-terrace-1340-west-28th-st-at-main-avenue-bridge-loc_6629877709_o.jpg', 'link': '/item/98518952/', 'title': 'Live here at low rent - Lakeview Terrace, 1340 West 28th St. at Main Avenue bridge'}
{'image': '/static/portals/free-to-use/public-domain/wpa-posters/be-careful-near-machinery-loc_6629877217_o.jpg', 'link': '/item/98518717/', 'title': 'Be careful near machinery'}
{'image': '/static/portals/free-to-use/public-domain/wpa-posters/the-national-parks-preserve-wild-life-loc_6629876663_o.jpg', 'link': '/item/98518597/', 'title': 'The national parks preserve wild life'}
{'image': '/static/portals/free-to-use/public-domain/wpa-posters/understanding-the-arts-loc_6629876317_o.jpg', 'link': '/item/98518272/', 'ti

In [8]:
len(collection_json['content']['set']['items'])

24

In [9]:
collection_json['content']['set']['items'][0].keys()

dict_keys(['image', 'link', 'title'])

In [13]:
collection_set_list = '/Users/julia.cave.arbanas/si676-2024-data/collection_project_WPA/ftu-WPA-set-list.csv'
headers = ['image','link','title']

with open(collection_set_list, 'w', encoding='utf-8', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=headers)
    writer.writeheader()
    for item in collection_json['content']['set']['items']:
        item['title'] = item['title'].rstrip()
        writer.writerow(item)
    print('wrote',collection_set_list)

wrote /Users/julia.cave.arbanas/si676-2024-data/collection_project_WPA/ftu-WPA-set-list.csv


In [11]:
import csv
import json
import requests

import os
from os.path import join

In [12]:
print(os.getcwd())

/Users/julia.cave.arbanas/si676-2024-data/collection-project


In [14]:
def regenerate_collection_list(collection_csv):
    """
    Reads a CSV file and returns the data as a dictionary.
    
    Parameters:
    collection_csv (str): The path to the CSV file

    Returns:
    dict: A dictionary where each key is a column header and each value is a list of column values.
    """

    coll_items = list()

    with open(collection_csv, 'r', newline='', encoding='utf-8') as f:
        data = csv.DictReader(f)

        for row in data:
            row_dict = dict()
            for field in data.fieldnames:
                row_dict[field] = row[field]
            coll_items.append(row_dict)

        return coll_items


In [16]:
collection_csv = os.path.join('/Users/julia.cave.arbanas/si676-2024-data/collection_project_WPA', 'ftu-WPA-set-list.csv')

collection_set_list = regenerate_collection_list(collection_csv)

In [17]:
collection_set_list[0]

{'image': '/static/portals/free-to-use/public-domain/wpa-posters/a-book-mark-would-be-better-loc_6629878315_o.jpg',
 'link': '/item/2011645389/',
 'title': 'A book mark would be better!'}

In [18]:
baseURL = 'https://www.loc.gov'
parameters = {
    'fo' : 'json'
}

In [19]:
import os
from os.path import join

base_directory = os.path.expanduser('~/si676-2024-data')
item_metadata_directory = join(base_directory, 'collection_project_WPA', 'item_metadata')

if os.path.isdir(item_metadata_directory):
    print(item_metadata_directory,'exists')
else:
    os.mkdir(item_metadata_directory)
    print('created',item_metadata_directory)

created /Users/julia.cave.arbanas/si676-2024-data/collection_project_WPA/item_metadata


In [20]:
item_count = 0
error_count = 0
file_count = 0

base_directory = os.path.expanduser('~/si676-2024-data')
item_metadata_directory = os.path.join(base_directory, 'collection_project_WPA', 'item_metadata')

if not os.path.exists(item_metadata_directory):
    os.makedirs(item_metadata_directory)
    print('Created directory:', item_metadata_directory)

data_directory = 'collection_project_WPA'
item_metadata_directory = 'item_metadata'
item_metadata_file_prefix = 'item_metadata'
json_suffix = '.json'

for item in collection_set_list:
    if item['link'] == 'link':
        continue
    if '?' in item['link']:
        resource_ID = item['link']
        short_ID = item['link'].split('/')[2]
        item_metadata = requests.get(baseURL + resource_ID, params={'fo':'json'})
        print('requested',item_metadata.url,item_metadata.status_code)
        if item_metadata.status_code != 200:
            print('requested',item_metadata.url,item_metadata.status_code)
            error_count += 1
            continue
        try:
            item_metadata.json()
        except:
            error_count += 1
            print('no json found')
            continue
        fout = os.path.join(base_directory,data_directory, item_metadata_directory, str(item_metadata_file_prefix + '-' + short_ID + json_suffix))
        with open(fout, 'w', encoding='utf-8') as json_file:
            json_file.write(json.dumps(item_metadata.json()['item']))
            file_count += 1
            print('wrote', fout)
        item_count += 1
    else:
        resource_ID = item['link']
        short_ID = item['link'].split('/')[2]
        item_metadata = requests.get(baseURL + resource_ID, params={'fo':'json'})
        print('requested',item_metadata.url,item_metadata.status_code)
        if item_metadata.status_code != 200:
            print('requested',item_metadata.url,item_metadata.status_code)
            error_count += 1
            continue
        try:
            item_metadata.json()
        except:
            error_count += 1
            print('no json found')
            continue
        fout = os.path.join(base_directory,data_directory, item_metadata_directory, str(item_metadata_file_prefix + '-' + short_ID + json_suffix))
        with open(fout, 'w', encoding='utf-8') as json_file:
            json_file.write(json.dumps(item_metadata.json()['item']))
            file_count += 1
            print('wrote', fout)
        item_count += 1

print('--- mini LOG ---')
print('items requested:',item_count)
print('errors:',error_count)
print('files written:',file_count)

requested https://www.loc.gov/item/2011645389/?fo=json 200
wrote /Users/julia.cave.arbanas/si676-2024-data/collection_project_WPA/item_metadata/item_metadata-2011645389.json
requested https://www.loc.gov/item/98518952/?fo=json 200
wrote /Users/julia.cave.arbanas/si676-2024-data/collection_project_WPA/item_metadata/item_metadata-98518952.json
requested https://www.loc.gov/item/98518717/?fo=json 200
wrote /Users/julia.cave.arbanas/si676-2024-data/collection_project_WPA/item_metadata/item_metadata-98518717.json
requested https://www.loc.gov/item/98518597/?fo=json 200
wrote /Users/julia.cave.arbanas/si676-2024-data/collection_project_WPA/item_metadata/item_metadata-98518597.json
requested https://www.loc.gov/item/98518272/?fo=json 200
wrote /Users/julia.cave.arbanas/si676-2024-data/collection_project_WPA/item_metadata/item_metadata-98518272.json
requested https://www.loc.gov/item/98518267/?fo=json 200
wrote /Users/julia.cave.arbanas/si676-2024-data/collection_project_WPA/item_metadata/item

In [21]:
base_directory = os.path.expanduser('~/si676-2024-data')
project_dir = 'collection_project_WPA'
files_dir = 'item_files'
metadata_dir = 'item_metadata'

files_loc = os.path.join(base_directory,project_dir,files_dir)
print('Checking for',files_loc)

# check directory
if os.path.isdir(files_loc):
    print('Files directory exists')
else:
    os.mkdir(files_loc)
    print('Created file directory:',files_loc)

Checking for /Users/julia.cave.arbanas/si676-2024-data/collection_project_WPA/item_files
Created file directory: /Users/julia.cave.arbanas/si676-2024-data/collection_project_WPA/item_files


In [24]:
import glob

In [23]:
import glob

base_directory = os.path.expanduser('~/si676-2024-data')

search_for_metadata_here = os.path.join(base_directory,project_dir,metadata_dir)

print(search_for_metadata_here)

metadata_file_list = glob.glob(search_for_metadata_here + '/*.json')

print(metadata_file_list)

/Users/julia.cave.arbanas/si676-2024-data/collection_project_WPA/item_metadata
['/Users/julia.cave.arbanas/si676-2024-data/collection_project_WPA/item_metadata/item_metadata-98507722.json', '/Users/julia.cave.arbanas/si676-2024-data/collection_project_WPA/item_metadata/item_metadata-96503125.json', '/Users/julia.cave.arbanas/si676-2024-data/collection_project_WPA/item_metadata/item_metadata-98508422.json', '/Users/julia.cave.arbanas/si676-2024-data/collection_project_WPA/item_metadata/item_metadata-98518272.json', '/Users/julia.cave.arbanas/si676-2024-data/collection_project_WPA/item_metadata/item_metadata-98517129.json', '/Users/julia.cave.arbanas/si676-2024-data/collection_project_WPA/item_metadata/item_metadata-93511155.json', '/Users/julia.cave.arbanas/si676-2024-data/collection_project_WPA/item_metadata/item_metadata-92509203.json', '/Users/julia.cave.arbanas/si676-2024-data/collection_project_WPA/item_metadata/item_metadata-98517179.json', '/Users/julia.cave.arbanas/si676-2024-da

In [25]:
item_image_urls = list()
count = 0

for item in metadata_file_list:
    with open(item, 'r', encoding='utf-8') as f:
        metadata = json.load(f)
        # noted this resource for working out index out of range errors: https://rollbar.com/blog/how-to-fix-python-list-index-out-of-range-error-in-for-loops/
        image_url_no = len(metadata['image_url'])
        image_url = metadata['image_url'][-1]
        item_image_urls.append(image_url)
        count += 1

print(f'Identified { str(count) } image URLs')



Identified 24 image URLs


In [26]:
item_image_urls

['https://tile.loc.gov/storage-services/service/pnp/cph/3f00000/3f05000/3f05100/3f05175v.jpg#h=1024&w=688',
 'https://tile.loc.gov/storage-services/service/pnp/cph/3g00000/3g04000/3g04200/3g04243v.jpg#h=1024&w=829',
 'https://tile.loc.gov/storage-services/service/pnp/cph/3b40000/3b48000/3b48700/3b48755v.jpg#h=1024&w=658',
 'https://tile.loc.gov/storage-services/service/pnp/cph/3b40000/3b48000/3b48700/3b48726v.jpg#h=1024&w=687',
 'https://tile.loc.gov/storage-services/service/pnp/cph/3b40000/3b48000/3b48800/3b48810v.jpg#h=1024&w=655',
 'https://tile.loc.gov/storage-services/service/pnp/cph/3b40000/3b49000/3b49000/3b49006v.jpg#h=1024&w=799',
 'https://tile.loc.gov/storage-services/service/pnp/cph/3b40000/3b48000/3b48900/3b48901v.jpg#h=1024&w=810',
 'https://tile.loc.gov/storage-services/service/pnp/cph/3b40000/3b48000/3b48800/3b48809v.jpg#h=1024&w=657',
 'https://tile.loc.gov/storage-services/service/pnp/cph/3b40000/3b49000/3b49000/3b49068v.jpg#h=1024&w=791',
 'https://tile.loc.gov/stora

In [27]:
collection_set_list_with_images = list()

for item in metadata_file_list:
    with open(item, 'r', encoding='utf-8') as item_info:
        item_metadata = json.load(item_info)

        item_metadata_dict = dict()
        item_metadata_dict['item_URI'] = item_metadata['id']
        try:
            item_metadata_dict['lccn'] = item_metadata['library_of_congress_control_number']
        except:
            item_metadata_dict['lccn'] = None
        item_metadata_dict['title'] = item_metadata['title']
        item_metadata_dict['image_URL_large'] = item_metadata['image_url'][-1]
        
        collection_set_list_with_images.append(item_metadata_dict)

print(collection_set_list_with_images[0])

{'item_URI': 'http://www.loc.gov/item/98507722/', 'lccn': '98507722', 'title': "In March read the books you've always meant to read", 'image_URL_large': 'https://tile.loc.gov/storage-services/service/pnp/cph/3f00000/3f05000/3f05100/3f05175v.jpg#h=1024&w=688'}


In [28]:
item_count = 0
error_count = 0
file_count = 0

img_file_prefix = 'img_'

base_directory = os.path.expanduser('~/si676-2024-data')

for item in collection_set_list_with_images:
        image_URL = item['image_URL_large']
        short_ID = item['item_URI'].split('/')[-2]
        print('... requesting',image_URL)
        item_count += 1

        r = requests.get(image_URL)
        if r.status_code == 200:
            img_out = os.path.join(base_directory,project_dir,files_dir,str(img_file_prefix + short_ID + '.jpg'))
            with open(img_out, 'wb') as file:
                file.write(r.content)
                print('Saved',img_out)
                file_count += 1


print('--- mini LOG ---')
print('files requested:',item_count)
print('errors:',error_count)
print('files written:',file_count)

... requesting https://tile.loc.gov/storage-services/service/pnp/cph/3f00000/3f05000/3f05100/3f05175v.jpg#h=1024&w=688
Saved /Users/julia.cave.arbanas/si676-2024-data/collection_project_WPA/item_files/img_98507722.jpg
... requesting https://tile.loc.gov/storage-services/service/pnp/cph/3g00000/3g04000/3g04200/3g04243v.jpg#h=1024&w=829
Saved /Users/julia.cave.arbanas/si676-2024-data/collection_project_WPA/item_files/img_96503125.jpg
... requesting https://tile.loc.gov/storage-services/service/pnp/cph/3b40000/3b48000/3b48700/3b48755v.jpg#h=1024&w=658
Saved /Users/julia.cave.arbanas/si676-2024-data/collection_project_WPA/item_files/img_98508422.jpg
... requesting https://tile.loc.gov/storage-services/service/pnp/cph/3b40000/3b48000/3b48700/3b48726v.jpg#h=1024&w=687
Saved /Users/julia.cave.arbanas/si676-2024-data/collection_project_WPA/item_files/img_98518272.jpg
... requesting https://tile.loc.gov/storage-services/service/pnp/cph/3b40000/3b48000/3b48800/3b48810v.jpg#h=1024&w=655
Saved /Us